# Purpose of this notebook:
The purpose of this notebook is to plot aggregate data collected thus far

Notes:

Experiment Done on 11-5-19 was done using Saline for the first 3 then CCAP (10^-7M) and then another saline

Use ctrl + click to select multiple options for dates and conditions

Some dates involved multiple concentrations of CCAP, in which case the CCAP will have two different line plots with the same color for that date (See individiual notebooks for more details)

# Load Required Packages

In [1]:
%matplotlib inline
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib.lines import Line2D
from matplotlib import pyplot as plt
from collections import OrderedDict
from IPython.display import display
# Widget stuff
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed, interactive

# Load Data

In [2]:
p = 'nusbaum_lab_data.csv'
df = pd.read_csv(p, index_col=0)

# Change it so that All pres and all posts aggregate as one condition
df.loc[df.Condition.isin(['Pre', 'Pre 0', 'Pre 1', 'Pre 2']), 'Condition'] = 'Pre'
df.loc[df.Condition.isin(['Post', 'Post 1', 'Post 2']), 'Condition'] = 'Post'
dataframe = df

# Make things used later in plots
dates=list(dataframe['Date'].unique())
Conditions = list(dataframe['Condition'].unique())
markers = Line2D.filled_markers[:10]
cmap=sns.color_palette('bright',10)
marker_d = dict(zip(sorted(df['Date'].unique()),markers))
color_d = dict(zip(sorted(df['Date'].unique()), cmap))

In [3]:
@widgets.interact(
    y_val=['Burst Duration (sec)', '# of Spikes',
           'Spike Frequency (Hz)', 'Instantaneous Period (sec)',
           'Instantaneous Frequency (Hz)','Duty Cycle'], 
    burst_num=(1, 300),
    neuron = ['LG', 'DG', 'GM'], 
    Condition = widgets.SelectMultiple(
        options=['Pre', 'CCAP', 'Unfed Hemo + CCAP', 'Post'],
        value=tuple(['Pre', 'CCAP', 'Unfed Hemo + CCAP', 'Post']),
        rows = 4,
        description='Condition',
        disabled=False),
    dates=widgets.SelectMultiple(
        options=df['Date'].unique(),
        value=tuple(df['Date'].unique()),
        rows=len(df['Date'].unique()),
        description='Dates',
        disabled=False
        ))


def plot(y_val='Burst Duration (sec)', burst_num=100, plot_lines=True, 
         title='Plot title', neuron='LG',dates=dates, Condition=('Pre'),
        ):
    global dataframe
    global marker_d
    global color_d
    _dataframe = dataframe[(dataframe['Burst#'] < burst_num)
                           &(dataframe['Neuron']==neuron)
                           #&(dataframe['Condition']==condition)
                           &(dataframe.Condition.isin(Condition))
                           &(dataframe.Date.isin(dates))]
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,5))
    for i,date in enumerate(_dataframe['Date'].unique()):
        data = _dataframe[_dataframe['Date']==date]
        plot = ax.scatter(x=data['Burst#'], 
                          y=data[y_val], 
                          label='{} LG {}'.format(date, neuron),
                          alpha=.5,
                          marker=marker_d[date],
                          color=color_d[date]
                         )
        if plot_lines:
            if data[data['Burst#']==1].shape[0] == 1:
                plot_ = ax.plot(data['Burst#'], 
                                data[y_val], color=color_d[date], ls='--',zorder=3, label='')
            elif data[data['Burst#']==1].shape[0] > 1:
                # Fix the issue here
                # This hassle and weird code is to prevent line connections between seperate
                # Experiments
                #x_data = find_consecutive_data(data['Burst#'])
                x_data = np.split(data['Burst#'],
                                  np.where(np.diff(data['Burst#']) != 1)[0] + 1)
                y_data = np.split(data[y_val], 
                                  np.where(np.diff(data['Burst#']) != 1)[0] + 1)
                for x,y in zip(x_data, y_data):
                    plot_ = ax.plot(x, y, color=color_d[date], ls='--',zorder=3, label='')
                
                

    y_label = ax.set_ylabel(y_val,fontsize=16)
    x_label = ax.set_xlabel('Burst #',fontsize=16)
    ax.set_title(title,fontsize=20)
    plt.legend(bbox_to_anchor=(1,1))
    plt.tight_layout()

interactive(children=(Dropdown(description='y_val', options=('Burst Duration (sec)', '# of Spikes', 'Spike Fre…

In [31]:
def find_consecutive_data(data, step=1):
    """Splits Data into a list of arrays where there is more than a single step of 1

    Parameters
    ----------
    data: np.array (of zeroes and ones, True or False, etc.)
          the data to split
    step: int, by default 1

    Returns
    -------
    list of split data
    """
    return np.split(data, np.where(np.diff(data) != step)[0] + 1)

def find_close_data(data, allowable_step=1):
    """Splits Data into a list of arrays where there a adjacent increase greater than allowable_step

    Parameters
    ----------
    data: np.array, the data to split
    step: int, by default 1

    Returns
    -------
    list of split data
    """
    return np.split(data, np.where(np.diff(data) > allowable_step)[0] + 1)